In [ ]:
# ==============================================================
# 02_WRI_IntersectionReports.py
# --------------------------------------------------------------
# Purpose
#   Identify which WRI Urban Land Use V1 rasters intersect with
#   your DEPRIMAP city-segment GPKGs, and write summary CSVs.
#
# Context
#   - WRI rasters were downloaded from GEE using:
#       01_WRI_DataDownload.js
#   - The raw GeoTIFFs are *not* included in the GitHub repo
#     (large size). Users should download them from GEE
#     following the Guzder-Williams et al. dataset instructions.
#   - Based on the CSV reports produced here, a *manual* step
#     is used to organise the subset of rasters into a folder
#     structure like:
#
#       PerCountry_Outputs/
#           angola/
#               <list of WRI rasters intersecting angola segments>
#           kenya/
#               ...
#
#     That PerCountry_Outputs tree is used in the next WRI
#     notebook for block-level summaries and comparisons.
#
# Inputs (not shipped in repo)
#   - TIF_DIR: folder containing downloaded WRI GeoTIFFs
#              from 01_WRI_DataDownload.js
#
# Outputs (CSV files, *included* in repo)
#   /intersect_reports/:
#       - raster_gpkg_pairs_all.csv
#            one row per (raster, gpkg) pair checked, with
#            intersection status and fraction of raster bbox.
#       - rasters_with_any_intersection.csv
#            one row per raster that intersects at least one
#            GPKG; includes count of intersecting GPKGs.
#       - raster_to_gpkg_list.csv
#            for each raster, the list of intersecting GPKG
#            filenames and count of GPKGs.
#       - pairwise_intersection_fraction.csv
#            (raster, gpkg) pairs with intersection fractions
#            sorted by raster and descending fraction.
#
# Method
#   1. For each WRI raster:
#        - build its bounding box polygon in raster CRS.
#   2. For each city-segment GPKG:
#        - read all layers, clean geometries, union per layer
#          and reproject to raster CRS.
#        - union across layers to get a single vector geometry.
#   3. Compute the intersection between raster bbox and the
#      union of the city-segment geometries.
#   4. Record whether they intersect and what fraction of the
#      raster bbox is covered.
#
# NOTE:
#   - Intersection is done at the *bounding box* level of the
#     rasters, which is sufficient for selecting candidate
#     rasters. Later notebooks do stricter masking.
# ==============================================================

In [ ]:
# STEP 1 — Imports & configuration
from pathlib import Path
import pandas as pd
import numpy as np
import rasterio as rio
from shapely.geometry import box
from shapely.ops import unary_union
import geopandas as gpd
import fiona

In [ ]:
# --------------------------------------------------------------
# >>> EDIT THESE TWO PATHS FOR YOUR ENVIRONMENT <<<
# --------------------------------------------------------------
# Folder with downloaded WRI Urban Land Use V1 GeoTIFFs
TIF_DIR = Path("../WRI/WRI_urban_landuse_v1")

# Folder with CSD city-segment GPKGs (Filtered 80% application)
GPKG_DIR = Path("../2_modelling/02_application/Filtered_80pct_allattributes")

# File patterns
RASTER_PATTERNS = ("*.tif", "*.tiff")
GPKG_PATTERN    = "*.gpkg"

print("WRI raster folder:", TIF_DIR.resolve())
print("CSD GPKG folder  :", GPKG_DIR.resolve())

In [ ]:
# STEP 2 — Helper functions
# --------------------------------------------------------------


def iter_files(root: Path, patterns):
    """Yield all files under `root` matching the given glob pattern(s)."""
    pats = patterns if isinstance(patterns, (list, tuple)) else [patterns]
    for pat in pats:
        yield from root.rglob(pat)


def raster_bbox_polygon(raster_path: Path):
    """Return (bbox_polygon, CRS) for the raster bounding box."""
    with rio.open(raster_path) as ds:
        l, b, r, t = ds.bounds
        raster_crs = ds.crs
    return box(l, b, r, t), raster_crs


def gpkg_union_geometry(gpkg_path: Path):
    """
    Read ALL layers in a GPKG, return:
        (layer_infos, layer_error)
    where layer_infos is a list of tuples:
        (layer_name, GeoDataFrame|None, note_str|None)

    - Layers with no CRS or read errors are recorded with gdf=None.
    - Valid layers are cleaned (non-empty, valid geometries).
    """
    try:
        layers = fiona.listlayers(gpkg_path)
    except Exception as e:
        return [], f"ERROR listing layers: {e}"

    layer_infos = []
    for lyr in layers:
        try:
            gdf = gpd.read_file(gpkg_path, layer=lyr)
            if gdf.empty or gdf.geometry.is_empty.all():
                continue
            if gdf.crs is None:
                # No CRS → cannot safely reproject; record and skip.
                layer_infos.append((lyr, None, "SKIPPED: no CRS"))
                continue
            # Clean invalid geoms
            gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notna()].copy()
            gdf["geometry"] = gdf.geometry.buffer(0)
            if gdf.empty:
                continue
            layer_infos.append((lyr, gdf, None))
        except Exception as e:
            layer_infos.append((lyr, None, f"ERROR reading layer: {e}"))

    return layer_infos, None


def reproject_geom_to_crs(gdf: gpd.GeoDataFrame, target_crs):
    """
    Reproject GeoDataFrame to target CRS and return a unary union geometry.
    """
    if gdf.crs == target_crs:
        return unary_union(gdf.geometry.values)
    else:
        gdf2 = gdf.to_crs(target_crs)
        return unary_union(gdf2.geometry.values)

In [ ]:
# STEP 3 — Compute intersections (bbox vs. vector union)
# --------------------------------------------------------------

# Index all GPKGs once; we will reproject per raster
gpkg_paths = sorted(iter_files(GPKG_DIR, GPKG_PATTERN))
print(f"Found {len(gpkg_paths)} GPKG files")

results = []  # one row per (raster, gpkg)

raster_paths = sorted(iter_files(TIF_DIR, RASTER_PATTERNS))
print(f"Found {len(raster_paths)} GeoTIFFs")

for i, tif in enumerate(raster_paths, 1):
    if i % 10 == 0:
        print(f"… {i}/{len(raster_paths)} rasters processed: {tif.name}")
    try:
        # Raster bbox & CRS
        r_bbox, r_crs = raster_bbox_polygon(tif)
        r_area = r_bbox.area

        for gpkg in gpkg_paths:
            layer_infos, layer_err = gpkg_union_geometry(gpkg)
            if layer_err:
                results.append({
                    "raster": str(tif),
                    "raster_name": tif.name,
                    "raster_crs": str(r_crs),
                    "gpkg": str(gpkg),
                    "gpkg_name": gpkg.name,
                    "gpkg_status": layer_err,
                    "layers_checked": 0,
                    "intersects": False,
                    "intersection_area": 0.0,
                    "intersection_fraction_of_raster_bbox": 0.0,
                })
                continue

            # Reproject & union all valid layers to raster CRS
            any_geom = None
            layers_ok = 0
            for lyr, gdf, note in layer_infos:
                if gdf is None:
                    # skipped or error
                    continue
                try:
                    geom_union = reproject_geom_to_crs(gdf, r_crs)
                except Exception:
                    continue
                if geom_union is None or geom_union.is_empty:
                    continue
                any_geom = geom_union if any_geom is None else any_geom.union(geom_union)
                layers_ok += 1

            if any_geom is None or any_geom.is_empty:
                # No usable geometry in this GPKG
                results.append({
                    "raster": str(tif),
                    "raster_name": tif.name,
                    "raster_crs": str(r_crs),
                    "gpkg": str(gpkg),
                    "gpkg_name": gpkg.name,
                    "gpkg_status": "NO_VALID_GEOMS",
                    "layers_checked": layers_ok,
                    "intersects": False,
                    "intersection_area": 0.0,
                    "intersection_fraction_of_raster_bbox": 0.0,
                })
                continue

            # Intersection: raster bbox vs vector union
            inter = r_bbox.intersection(any_geom)
            inter_area = 0.0 if inter.is_empty else float(inter.area)
            intersects = inter_area > 0.0
            frac = 0.0 if r_area == 0 else inter_area / r_area

            results.append({
                "raster": str(tif),
                "raster_name": tif.name,
                "raster_crs": str(r_crs),
                "gpkg": str(gpkg),
                "gpkg_name": gpkg.name,
                "gpkg_status": "OK",
                "layers_checked": layers_ok,
                "intersects": intersects,
                "intersection_area": inter_area,
                "intersection_fraction_of_raster_bbox": frac,
            })

    except Exception as e:
        results.append({
            "raster": str(tif),
            "raster_name": tif.name,
            "raster_crs": None,
            "gpkg": None,
            "gpkg_name": None,
            "gpkg_status": f"RASTER_ERROR: {e}",
            "layers_checked": 0,
            "intersects": False,
            "intersection_area": None,
            "intersection_fraction_of_raster_bbox": None,
        })

res_df = pd.DataFrame(results)
print(f"\nRows in result table: {len(res_df)}")

In [ ]:
# STEP 4 — Summaries: which rasters intersect which GPKGs?
# --------------------------------------------------------------

# Filter only intersection hits with valid GPKGs
hits = res_df[res_df["intersects"] & (res_df["gpkg_status"] == "OK")].copy()

# 4a) List of rasters that intersect *any* GPKG
rasters_with_hits = (
    hits.groupby(["raster", "raster_name", "raster_crs"])
        .agg(n_gpks=("gpkg_name", "nunique"))
        .reset_index()
        .sort_values("raster_name")
)

print("\nSample: rasters with at least one intersection")
display(rasters_with_hits.head(20))
print(f"Rasters with at least one intersection: {len(rasters_with_hits)}")

# 4b) For each raster, list the intersecting GPKGs
raster_to_gpkgs = (
    hits.groupby(["raster", "raster_name"])
        .agg(
            gpkgs=("gpkg_name", lambda s: sorted(set(s))),
            n_gpks=("gpkg_name", "nunique"),
        )
        .reset_index()
        .sort_values(["n_gpks", "raster_name"], ascending=[False, True])
)

print("\nSample: raster → list of intersecting GPKGs")
display(raster_to_gpkgs.head(20))

# 4c) For each (raster, gpkg) pair, show intersection fraction of raster bbox
pairs_ranked = (
    hits[["raster_name", "gpkg_name", "intersection_fraction_of_raster_bbox"]]
    .sort_values(["raster_name", "intersection_fraction_of_raster_bbox"],
                 ascending=[True, False])
)

print("\nSample: (raster, gpkg) pairs ranked by intersection fraction")
display(pairs_ranked.head(30))


# STEP 5 — Save CSVs (these are shipped in the repo)
# --------------------------------------------------------------

OUT = "intersect_reports"
OUT.mkdir(parents=True, exist_ok=True)

res_df.to_csv(OUT / "raster_gpkg_pairs_all.csv", index=False)
rasters_with_hits.to_csv(OUT / "rasters_with_any_intersection.csv", index=False)
raster_to_gpkgs.to_csv(OUT / "raster_to_gpkg_list.csv", index=False)
pairs_ranked.to_csv(OUT / "pairwise_intersection_fraction.csv", index=False)

print("\n✅ Saved intersection reports to:", OUT)

# NOTE FOR README:
#   Based on these CSVs, the subset of relevant WRI rasters were
#   manually copied/moved into:
#       <some_root>/WRI/PerCountry_Outputs/<country>/
#   and only those curated rasters are used in the next WRI
#   comparative analysis notebook.